In [1]:
import cv2
import pytesseract
import json

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [6]:
# 📽 Input video
video_path = "Vid_source/VD_0001.mp4"

# 🎥 Open video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
fps = int(fps)
frame_data = {}
frame_num = 0

cur_frame = 0
frame_milestone = 25
print(fps)

25


In [7]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    cur_frame += 1
    if cur_frame % frame_milestone != 0:
        continue

    h, w, _ = frame.shape
    boxes = pytesseract.image_to_data(frame, output_type=pytesseract.Output.DICT, lang='vie')

    seconds = (cur_frame - 1) // fps + 1 
    data_for_frame = []

    for i in range(len(boxes['text'])):
        if int(boxes['conf'][i]) > 60 and boxes['text'][i].strip():
            x = boxes['left'][i]
            y = boxes['top'][i]
            bw = boxes['width'][i]
            bh = boxes['height'][i]
            text = boxes['text'][i].strip()

            if text is None or text == "":
                continue

            # Normalize coordinates (values from 0 to 1)
            data_for_frame.append({
                "bbox": [x / w, y / h, bw / w, bh / h],
                "text": text
            })

    if data_for_frame:
        frame_data[str(seconds)] = data_for_frame
        # frame_num += 1

cap.release()

In [10]:
# 💾 Save to JSON
ratio = fps / frame_milestone

output = {
    "fps": fps,
    "data": frame_data,
    "ratio" : ratio
}

with open("ocr_data.json", "w") as f:
    json.dump(output, f, indent=2)

print("✅ OCR data saved to ocr_data.json")

✅ OCR data saved to ocr_data.json
